<B>Introduction/Business Problem


St. Petersburg is a small but growing city in central Florida, situated directly to the west of Tampa on the gulf coast. It is the 78th largest city in the U.S. and the fifth-largest city in Florida by population, and Tampa Bay (consisting of both Tampa & St. Petersburg) represents the second-most populated region in the state.

For many years, St. Pete was known as "God's waiting room" - an unflattering nickname for a city mostly known as a retirement destination for northerners tired of long winters. More recently, however, the city has attempted to capitalize on its long stretches of gulf coast beaches and its Guiness world record for most consecutive days of sunshine by re-branding itself "The Sunshine City." Today, the city draws more than 15 million annual visitors and boasts a revitalized downtown with an increasingly young and hip vibe. In addition, the 2017 ACS estimates the city population to be 41.2% ages 25-54, slightly younger than the national average of 39.2%.

This mix of growing tourism, a large base of year-round residents, and a yearly average of 360+ days of sunshine represents a strong location base for those looking to enter the hospitality industry. Specifically, our stakeholders are interested in finding the strongest potential locations for an ice cream shop in St. Petersburg. 

We will approach this problem by leveraging Foursquare location data for the city of St. Petersburg, taking into consideration several factors that would potentially impact traffic to an ice cream shop. Positive factors include the presence of parks, restaurants, schools, and entertainment venues, while the presence of competing ice cream shops is a major negative. We will analyze, cluster, and visualize this data with the hopes of isolating areas with unrealized or underutilized potential. 

<B>Data

First, we will need to define the geographical bounds for our search. Through an internet search, we have found an existing GEOJSon file that defines the bounds of 82 distinct neighborhoods within the city of St. Petersburg. From this file, we will extract the names of each neighborhood and calculate a crude "midpoint" for each neighborhood by locating the min/max values for latitute and longitude and then averaging them to find a centerpoint. We will then use that centerpoint to conduct searches to the Foursquare API's "explore" endpoint for relevant keywords including, but not limited to: ice cream, restaurant, park, school, and entertainment. We will then visualize the results and cluster neighborhoods to narrow down the candidates for our search. We will produce various visualizations and experiment with various metrics that could be useful including total number of venues found, the locations of competing ice cream shops, and the ratio of restaurants/entertainment venues to ice cream shops in each neighborhood.

<B>Exploration of St. Petersburg GEOJson file

In [1]:
import folium
import pandas as pd
from folium.features import GeoJsonPopup
import json

print("Modules loaded.")

Modules loaded.


In [4]:
#Load the GeoJSON file as a string and process the JSON
stpeteraw = open('stpete.geojson', 'r')
stpete = json.load(stpeteraw)

neighborhoods=[]
centlats=[]
centlongs=[]
#Find the names and centers of each neighborhood and append each to the lists initialized above
for place in stpete['features']:
    latmin = None
    latmax = None
    longmin = None
    longmax = None
    for latlong in place['geometry']['coordinates'][0]:
        if latmin == None or latlong[1] < latmin:
            latmin = latlong[1]
        if latmax == None or latlong[1] > latmax:
            latmax = latlong[1]
        if longmin == None or latlong[0] < longmin:
            longmin = latlong[0]
        if longmax == None or latlong[0] > longmax:
            longmax = latlong[0]
    centlat = (latmin + latmax)/2
    centlong = (longmin + longmax)/2
    neighborhoods.append(place['properties']['Name'])
    centlats.append(centlat)
    centlongs.append(centlong)

#Create a new dataframe utilizing the data extracted from the GeoJSON    
df = pd.DataFrame({'Neighborhood': neighborhoods, 'centerlat': centlats, 'centerlong': centlongs})
df.head()

,Neighborhood,centerlat,centerlong
0,Pinellas Point,27.706842,-82.660546
1,Wildwood Heights,27.758676,-82.667071
2,Sunset Drive South,27.768898,-82.743095
3,Brighton Bay,27.876057,-82.626393
4,Barclay Estates,27.852065,-82.651044


<B>Now that we have a dataframe of neighborhoods and the latitude/longitude of their centerpoints, we can map and inspect the results.

In [5]:
#initialize map and add the dataframe
world_map = folium.Map(location=[27.79,-82.66], zoom_start=11, tiles='OpenStreetMap')
choropleth = folium.Choropleth(
    geo_data=stpete,
#    data=df,
#    columns=['Country', 'Total'],
#     key_on='features.properties.Name',
    highlight=True,
#    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
#    legend_name='x'
).add_to(world_map)

#loop through neighborhoods dataframe and add a marker for the center of each neighborhood
for neigh, lat, lng, in zip(df.Neighborhood, df.centerlat, df.centerlong):
    world_map.add_child(
        folium.Circle(
            [lat, lng],
            radius=10,
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6,
            popup=neigh
        )
    )

#stylize the visual to look nicer and add mouseover tooltips to display neighborhood names
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
NIL = folium.features.GeoJson(
    stpete,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Name'],
        labels=False,
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
world_map.add_child(NIL)
world_map.keep_in_front(NIL)
folium.LayerControl().add_to(world_map)
world_map

This looks pretty good. In the coming week, we will use these midpoints to search the Foursquare API and start analyzing venues and neighborhoods.